In [1]:
import re
from collections import defaultdict

import morfeusz2 as morfeusz2
import numpy as np
from tqdm import tqdm
from utils import score_documents, prepare_question, scaled_editdist
from itertools import product

In [2]:
with open("../List_2/outputs/indexed_contents.txt", "r") as f:
    content_lines = f.readlines()

with open("../List_2/outputs/indexed_titles.txt", "r") as f:
    titles_lines = f.readlines()

morph = morfeusz2.Morfeusz()

In [3]:
def get_definition_from_wiki(content):
    i = 1
    sents = re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', content)
    definition = str(sents[0])
    while len(definition) < 20 and len(sents) > i + 1:
        i+=1
        definition += str(sents[i])

    return definition

def get_definition_from_question(question):
    definition = []
    beginnings = ["Co oznacza", "Co po polsku oznacza", "Jak miał na imię", "Jak miała na imię", "Jak nazywa się", "Jak w", "Jak z", "Jak brzmi"]
    for beginning in beginnings:
        l = len(beginning)
        if question[:l] == beginning:
            definition = question[l:]
            break
    if len(definition) > 0:
        definition = definition.split()
        definition = [w for w in definition if not ("oznacza" in w or "nazywa" in w)]
        definition = " ".join(definition)
        definition = definition[:-1]
    return definition

def preprocess_definition(definition, only_first = True):
    if len(definition) == 0:
        return set()
    definition = definition.strip().split()
    definition = [w for w in definition if w.isalpha() and len(w) > 2]
    current_idx = 0
    current_def = []
    analysis = morph.analyse(" ".join(definition))
    if only_first:
        for i, j, interp in analysis:
            if i == current_idx:
                current_def.append(interp[1].partition(":")[0])
                current_idx += 1
    else:
        for i, j, interp in analysis:
            current_def.append(interp[1].partition(":")[0])
    return " ".join(definition), set(current_def)

def prepare_rare_words(prepared_definitions, min, max):
    word_freq = defaultdict(int)
    for definition in tqdm(prepared_definitions):
        for word in definition[1]:
            word_freq[word] +=1
    rare_words = set([k for k, v in word_freq.items() if min < v < max])
    rare_words_freq = {k:v for k, v in word_freq.items() if min < v < max}
    return rare_words, rare_words_freq

def filter_definitions(definitions, titles, definition_features):
    filtered_definitions, filtered_titles, filtered_definition_features = [], [], []
    for i, features in enumerate(definition_features):
        if len(features) > 0:
            filtered_definitions.append(definitions[i])
            filtered_titles.append(titles[i])
            filtered_definition_features.append(features)
    return filtered_definitions, filtered_titles, filtered_definition_features

def find_similar_definitions(definition, titles, definitions, definition_features, return_titles=False):
    max_match = 0
    best_definitions_idx = []
    definition = preprocess_definition(definition, only_first=False)[1]
    for i, features in enumerate(definition_features):
        match = len(definition & features)
        if match == max_match:
            best_definitions_idx.append(i)
        if match > max_match:
            best_definitions_idx = [i]
            max_match = match
    if return_titles:
        return [titles[i] for i in best_definitions_idx]
    else:
        return [definitions[i][0] for i in best_definitions_idx]

def answer(definition, titles, definition_features, rare_words_freq):

    definition = preprocess_definition(definition, only_first=False)[1]
    max_score = 0
    best_definitions_idx = []
    for i, features in enumerate(definition_features):
        match = definition & features
        score = len(match)
        if score == max_score:
            best_definitions_idx.append((i, match))
        if score > max_score:
            best_definitions_idx = [(i, match)]
            max_score = score
    min_score = np.inf
    best_idx = 0
    for candidate_idx, candidate_match in best_definitions_idx:
        score = 0
        for word in candidate_match:
            score += rare_words_freq[word]
        if score < min_score:
            min_score = score
            best_idx = candidate_idx
    return titles_lines[titles[best_idx]]

In [4]:
def answer_mod(question, weights, threshold, titles, definitions, definition_features):
    yes_or_no = question.split()[0] == "Czy"
    definition = get_definition_from_question(question)
    if len(definition) > 0:
        candidates = find_similar_definitions(definition, titles, definitions, definition_features, return_titles=True)
    else:
        candidates = []
    _, question = prepare_question(question, morph)
    question = [token.lower() for token in question if len(token) > 1]
    while question:
        query = ' '.join(q for q in question)
        results = score_documents(query, weights, morph, candidates)
        search_results = list(results.keys())[:10]
        print(search_results)
        search_scores = list(results.values())[:10]
        search_results = zip(search_results, search_scores)
        for result, score in search_results:
            if yes_or_no:
                print(score / len(question))
                if score / len(question) < threshold:
                    return "nie"
                else:
                    return "tak"
            title = titles_lines[result]
            print(title)
            _, res_tokens = prepare_question(title, morph)
            to_remove = False
            # for t1, t2 in product(res_tokens, question):
            #     if scaled_editdist(t1, t2) <= 0.4:
            #         to_remove = True
            # if not to_remove:
            paren_index = title.find('(')
            if paren_index != -1:
                title = title[:paren_index]
            return title

        # if answer not found, remove first token of query
        del question[0]

    return 'nie mam pojęcia, sorry \n'

In [5]:
prepared_definitions = []
prepared_titles = []
for i in tqdm(range(len(content_lines))):
    current_def = preprocess_definition(get_definition_from_wiki(content_lines[i]))
    if len(current_def[1]) > 3:
        prepared_definitions.append(current_def)
        prepared_titles.append(i)

100%|██████████| 1208362/1208362 [03:58<00:00, 5057.60it/s]


In [6]:
rare_words, rare_words_freq = prepare_rare_words(prepared_definitions, 0, 1000)
definition_features = []
for definition in tqdm(prepared_definitions):
    definition_features.append(rare_words & definition[1])

100%|██████████| 1116454/1116454 [00:05<00:00, 220529.63it/s]


In [7]:
filtered_definitions, filtered_titles, filtered_definition_features = filter_definitions(prepared_definitions, prepared_titles, definition_features)

In [8]:
%%time
find_similar_definitions("szybki obrót wykonywany na palcach jednej nogi", filtered_titles, filtered_definitions, filtered_definition_features)

CPU times: user 338 ms, sys: 15 µs, total: 338 ms
Wall time: 337 ms


['ATW ATW ang around the world pot dookoła świata element techniki piłkarskiej polegający wykonaniu nogą pełnego obrotu nad piłką trakcie żonglowania',
 'Zespół bolesnych nóg ruchów palców Zespół bolesnych nóg ruchów palców ang painful toes syndrome rzadki zespół chorobowy objawiający się bólem kończyn dolnych mimowolnymi ruchami stóp lub palców',
 'Komendy polecenia Taekwondo ANPALMOK promieniowa część przedramienia strony kciuka ANURO wewnątrz ANJOT usiąść komenda przód JOOMUK powierzchnia uderzeniowa przedniej części pięści głowy dalsze kości śródręcza APCHA BUSIGI kopnięcie przodu APCHA OLLIGI blok stopą przodu góry noga prosta kolanie również wymach nogi przodu APKUMCHI powierzchnia uderzeniowa stopy którą tworzą główki dalsze kości śródstopia BAKATPALMOK łokciowa część przedramienia strony palca małego',
 'Wymyk Wymyk przejście półzwisu zwisu podporu przodem tyłem przez przeniesienie nóg środka ciężkości ciała ponad osią chwytu ramion obrót ciała wokół osi poprzecznej nóg przód',

In [9]:
find_similar_definitions("szybki obrót wykonywany na palcach jednej nogi", filtered_titles, filtered_definitions, filtered_definition_features, return_titles=True)

[247902, 595270, 734333, 804901, 842587, 1179155]

In [10]:
%%time
find_similar_definitions("zbiór wszystkich punktów płaszczyzny znajdujący się w jednakowej odległości od jednego punktu", filtered_titles, filtered_definitions, filtered_definition_features)

CPU times: user 390 ms, sys: 3.91 ms, total: 394 ms
Wall time: 393 ms


['Układ heksagonalny Układ heksagonalny układ krystalograficzny którym trzy czterech osi leżą jednej płaszczyźnie mają jednakową długość kąt między nimi wynosi',
 'Fala płaska Fala płaska jest fala której powierzchnie falowe powierzchnie jednakowej fazie tworzą równoległe siebie linie proste gdy fala rozchodzi się powierzchni lub płaszczyzny gdy rozchodzi się przestrzeni trójwymiarowej']

In [11]:
# 13 / 189
with open('../List_2/data/pytania.txt', 'r') as questions:
    with open('t2_odpowiedzi_tylko_wektory.txt', 'w') as f_out:
        line = questions.readline()
        while line:
            definition = get_definition_from_question(line.strip())
            if len(definition) > 0:
                a = answer(definition, filtered_titles, filtered_definition_features, rare_words_freq)
                print(a)
                f_out.write(a.strip()+' \n')
            else:
                f_out.write("Nie o definicję" +' \n')
            line = questions.readline()

ASCII 

Twierdzenie Ptolemeusza 

Pierwiastek chemiczny 

Staw eliptyczny 

Buszyzm 

Bazylika konkatedralna Świętej Trójcy w Chełmży 

Lista niematerialnego dziedzictwa kulturowego wymagającego pilnej ochrony 

Faryngalizacja 

Kuba 

James Bond 

Świnka (herb szlachecki) 

Bambrzy 

Dekiel 

Runek (Beskid Sądecki) 

Bitwa pod Oliwą 

Ēostre 

Set (gra) 

Katoda 

SST Pod Solniskiem 

AWK 

Rycerze Okrągłego Stołu 

Ryszard Natusiewicz 

Zaćma 

Metr bieżący 

Gryf 

Pozycja birmańska 

Karawela 

Cohen Barbarzyńca 

Astrid Lindgren 

Choroba Heinego-Medina 

Jesioniki 

Ć 

Abu Kir 

Oscypek 

Teologia sukcesu 

Cyjon rudy 

Zawias 

Raut (ornament) 

Mikrobiologia 

Łubin żółty 

Złożenie funkcji 

Pacynka 

Jesioniki 

Sunyer I 

Binegacja 

Ćwierćnuta 

Ania z Zielonego Wzgórza (seria) 

Lądowisko Nieborów 

Cyrulik sewilski (opera) 

Tendrzak 

Aerobus (kolejka) 

Deklinacja magnetyczna 

Azerbejdżan 

Rybka zwana Wandą 

Mora (tkanina) 

AWK 

Dermatologia 

Łódzkie Spotkania z 

KeyboardInterrupt: 

In [12]:
with open('../List_2/data/pytania.txt', 'r') as questions:
    with open('t2_odpowiedzi_mieszane.txt', 'w') as f_out:
        line = questions.readline()
        while line:
            a = answer_mod(line.strip(), weights=[1, 1, 0.3, 0.3, 2], threshold=0.5, titles=filtered_titles, definitions=filtered_definitions, definition_features=filtered_definition_features)
            print(a)
            f_out.write(a.strip()+' \n')
            line = questions.readline()

[11569, 10159, 11572, 11576, 11579, 12827, 12881, 12882, 12883, 15824]
Alfa 

Alfa 

[47598, 13205, 15916, 1072840, 696166, 789530, 15205, 542025, 15915, 599494]
Okrąg dziewięciu punktów 

Okrąg dziewięciu punktów 

[]
[]
[]
[]
[]
[]
[]
[]
nie mam pojęcia, sorry 

[]
[]
[]
[]
[]
[]
[]
nie mam pojęcia, sorry 

[]
[]
[]
[]
[]
[]
[]
[]
nie mam pojęcia, sorry 

[]
[]
[]
[]
nie mam pojęcia, sorry 

[]
[]
[]
[]
[]
[]
[]
[]
nie mam pojęcia, sorry 

[]
[]
[]
[]
[]
[]
[]
nie mam pojęcia, sorry 

[]
[]
[]
[]
[]
[]
nie mam pojęcia, sorry 

[]
[]
[]
[]
nie mam pojęcia, sorry 

[]
[]
[]
[]
[]
[]
nie mam pojęcia, sorry 

[3066, 15943, 941131, 15027, 765295, 807455, 808056, 968930, 47106, 215942]
Pierwiastek chemiczny 

Pierwiastek chemiczny 

[]
[]
[]
[]
[]
[]
[]
nie mam pojęcia, sorry 

[]
[]
[]
[]
[]
[]
[]
[]


KeyboardInterrupt: 